In [1]:
#!pip install requests

In [2]:
import datetime
import requests
import base64
from _kluce import client_id, client_secret
from urllib.parse import urlencode

In [8]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64encoded string 
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret ==None:
            raise Exception("You must set client_id a client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": 'client_credentials'
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self. get_token_data()
        token_header = self.get_token_header()
        r = requests.post(token_url, data=token_data, headers=token_header)
        if r.status_code not in range(200, 299):
            raise Exception("Authentification failed!")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expire_in = data['expires_in']         # seconds
        expires = now + datetime.timedelta(seconds=expire_in)
        self.access_token = access_token
        self.access_token_expires = expires
        access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires<now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resources_headers(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resources(self, lookup_id, resource_type='albums', version = 'v1'):
        base_url = "https://api.spotify.com/v1/"
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resources_headers()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resources(_id, resource_type = 'albums', version = 'v1')
    
    def get_artist(self, _id):
        return self.get_resources(_id, resource_type = 'artists', version = 'v1')
    
    def search(self, query, search_type='artist'):
        headers = self.get_resources_headers()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})

        lookup_url = f"{endpoint}?{data}"

        r = requests.get(lookup_url, headers = headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [9]:
spotify = SpotifyAPI(client_id, client_secret)

In [12]:
spotify.search("A lannister always pays his debts", search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=A+lannister+always+pays+his+debts&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
       'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
       'id': '1hCkSJcXREhrodeIHQdav8',
       'name': 'Ramin Djawadi',
       'type': 'artist',
       'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
    

In [13]:
spotify.get_artist("1hCkSJcXREhrodeIHQdav8")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
 'followers': {'href': None, 'total': 530466},
 'genres': ['german soundtrack',
  'orchestral soundtrack',
  'scorecore',
  'soundtrack',
  'video game music'],
 'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
 'id': '1hCkSJcXREhrodeIHQdav8',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb91e7dd670bc7d3ec7d1cbc6f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517491e7dd670bc7d3ec7d1cbc6f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17891e7dd670bc7d3ec7d1cbc6f',
   'width': 160}],
 'name': 'Ramin Djawadi',
 'popularity': 66,
 'type': 'artist',
 'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}

In [15]:
spotify.get_album('41zMFsCjcGenYKVJYUXU2n')

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
   'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
   'id': '1hCkSJcXREhrodeIHQdav8',
   'name': 'Ramin Djawadi',
   'type': 'artist',
   'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'J